Datos de los taxis verdes de New York City 

In [1]:
import pandas as pd


In [2]:
from pathlib import Path

Cargo un arhivo .parquet para ver su composición 

In [47]:
df_taxiverde=pd.read_parquet('green_tripdata_2023-04.parquet')

In [48]:
df_taxiverde.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65392 entries, 0 to 65391
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               65392 non-null  int32         
 1   lpep_pickup_datetime   65392 non-null  datetime64[ns]
 2   lpep_dropoff_datetime  65392 non-null  datetime64[ns]
 3   store_and_fwd_flag     60910 non-null  object        
 4   RatecodeID             60910 non-null  float64       
 5   PULocationID           65392 non-null  int32         
 6   DOLocationID           65392 non-null  int32         
 7   passenger_count        60910 non-null  float64       
 8   trip_distance          65392 non-null  float64       
 9   fare_amount            65392 non-null  float64       
 10  extra                  65392 non-null  float64       
 11  mta_tax                65392 non-null  float64       
 12  tip_amount             65392 non-null  float64       
 13  t

Creo una referencia de los atributos de cada columna para solo procesar las columnas relevantes para el proyecto

In [ ]:
VendorID: Identificador del proveedor de servicio de taxi.
lpep_pickup_datetime: Fecha y hora de recogida del pasajero.
lpep_dropoff_datetime: Fecha y hora de finalización del viaje.
store_and_fwd_flag: Indicador de si el vehículo almacenó los datos del viaje antes de enviarlos.
RatecodeID: Identificador del código de tarifa utilizado para el viaje.
PULocationID: Identificador de la ubicación de recogida.
DOLocationID: Identificador de la ubicación de destino.
passenger_count: Cantidad de pasajeros en el viaje.
trip_distance: Distancia recorrida en el viaje.
fare_amount: Tarifa básica del viaje.
extra: Cargo adicional en la tarifa.
mta_tax: Impuesto MTA en la tarifa.
tip_amount: Cantidad de propina.
tolls_amount: Monto de peajes.
ehail_fee: Tarifa de solicitud de viaje a través de una aplicación (ehail).
improvement_surcharge: Cargo de mejora en la tarifa.
total_amount: Monto total pagado por el pasajero.
payment_type: Tipo de pago utilizado en el viaje.
trip_type: Tipo de viaje (quizás ida o ida y vuelta).
congestion_surcharge: Cargo por congestión en la tarifa (puede depender de la ubicación y regulaciones locales)

Defino la carpeta donde se encuntran los archivos .parquet

In [33]:
folder_pathv = 'C:\\Users\\yopab\\Downloads\\Datasets\\dfexternos'

Obtengo la lista de archivos .parquet en la carpeta

In [34]:

parquet_filesv = Path(folder_pathv).glob('*.parquet')

Filtro los registros que correspondan al año 2021

In [5]:
parquet_files_2021 = [file for file in parquet_filesv if 'green_tripdata_2021' in file.name]

Inicializo una lista para almacenar los df de cada archivo

In [6]:
data_frames_2021 = []

Cargo y combino los df 

In [7]:
for file in parquet_files_2021:
    df = pd.read_parquet(file)
    data_frames_2021.append(df)

Combino los df en uno solo

In [8]:
combined_df_2021 = pd.concat(data_frames_2021, ignore_index=True)

Convierto las columnas de fechas a datetime

In [10]:
combined_df_2021['lpep_pickup_datetime'] = pd.to_datetime(combined_df_2021['lpep_pickup_datetime'])
combined_df_2021['lpep_dropoff_datetime'] = pd.to_datetime(combined_df_2021['lpep_dropoff_datetime'])

Agrego una columna para el día

In [11]:
combined_df_2021['pickup_date'] = combined_df_2021['lpep_pickup_datetime'].dt.date

Agrupo por día y sumo los valores diarios que van a ser relevantes para el modelo

In [12]:
daily_totals = combined_df_2021.groupby('pickup_date').agg({
    'trip_distance': 'sum',
    'fare_amount': 'sum',
    'tip_amount': 'sum',
    'PULocationID': 'sum',# Identificador de la ubicación de recogida.
    'DOLocationID': 'sum',#Identificador de la ubicación de destino.
    'total_amount': 'sum',#Monto total pagado por el pasajero.
    'payment_type': 'sum', #Tipo de pago utilizado en el viaje.

}).reset_index()

Guardo el el df como archivo CSV

In [13]:
daily_totals.to_csv('totales_diarios_tverde_2021.csv', index=False)

compruebo que todo haya salido bien 

In [14]:
totalesverdes=pd.read_csv('totales_diarios_tverde_2021.csv')

In [15]:
totalesverdes

,pickup_date,trip_distance,fare_amount,tip_amount,PULocationID,DOLocationID,total_amount,payment_type
0,2008-12-31,5.17,23.00,1.00,697,796,29.35,5.0
1,2009-01-01,39.60,155.50,0.00,3502,3425,174.35,36.0
2,2010-09-23,5.05,112.50,0.01,679,617,115.21,7.0
3,2020-12-31,1.99,8.00,0.00,168,75,9.30,2.0
4,2021-01-01,5337.77,21609.32,2005.33,105570,132339,25312.43,917.0
...,...,...,...,...,...,...,...,...
364,2021-12-27,340031.62,36129.76,2751.88,218449,289244,43424.99,2829.0
365,2021-12-28,65687.52,39364.18,3107.30,220783,305345,47391.23,2857.0
366,2021-12-29,133273.99,39600.44,3480.64,230036,310732,48213.85,2930.0
367,2021-12-30,138802.01,44464.51,3802.51,250853,331564,53698.42,3182.0


El mismo proceso se repite con el año 2022 y 2023 opteniendo los df separdos 

In [22]:
parquet_files_2022 = [file for file in parquet_filesv if 'green_tripdata_2022' in file.name]

In [23]:
parquet_files_2022

[WindowsPath('C:/Users/yopab/Downloads/Datasets/dfexternos/green_tripdata_2022-01.parquet'),
 WindowsPath('C:/Users/yopab/Downloads/Datasets/dfexternos/green_tripdata_2022-02.parquet'),
 WindowsPath('C:/Users/yopab/Downloads/Datasets/dfexternos/green_tripdata_2022-03.parquet'),
 WindowsPath('C:/Users/yopab/Downloads/Datasets/dfexternos/green_tripdata_2022-04.parquet'),
 WindowsPath('C:/Users/yopab/Downloads/Datasets/dfexternos/green_tripdata_2022-05.parquet'),
 WindowsPath('C:/Users/yopab/Downloads/Datasets/dfexternos/green_tripdata_2022-06.parquet'),
 WindowsPath('C:/Users/yopab/Downloads/Datasets/dfexternos/green_tripdata_2022-07.parquet'),
 WindowsPath('C:/Users/yopab/Downloads/Datasets/dfexternos/green_tripdata_2022-08.parquet'),
 WindowsPath('C:/Users/yopab/Downloads/Datasets/dfexternos/green_tripdata_2022-09.parquet'),
 WindowsPath('C:/Users/yopab/Downloads/Datasets/dfexternos/green_tripdata_2022-10.parquet'),
 WindowsPath('C:/Users/yopab/Downloads/Datasets/dfexternos/green_tripd

In [24]:
data_frames_2022 = []

In [25]:
for file in parquet_files_2022:
    df = pd.read_parquet(file)
    data_frames_2022.append(df)

In [26]:
combined_df_2022 = pd.concat(data_frames_2022, ignore_index=True)

In [27]:
# Convertir las columnas de fechas a datetime
combined_df_2022['lpep_pickup_datetime'] = pd.to_datetime(combined_df_2022['lpep_pickup_datetime'])
combined_df_2022['lpep_dropoff_datetime'] = pd.to_datetime(combined_df_2022['lpep_dropoff_datetime'])

In [28]:
combined_df_2022['pickup_date'] = combined_df_2022['lpep_pickup_datetime'].dt.date

In [29]:
daily_totals = combined_df_2022.groupby('pickup_date').agg({
    'trip_distance': 'sum',
    'fare_amount': 'sum',
    'tip_amount': 'sum',
    'PULocationID': 'sum',# Identificador de la ubicación de recogida.
    'DOLocationID': 'sum',#Identificador de la ubicación de destino.
    'total_amount': 'sum',#Monto total pagado por el pasajero.
    'payment_type': 'sum', #Tipo de pago utilizado en el viaje.

}).reset_index()

In [30]:
daily_totals.to_csv('totales_diarios_tverde_2022.csv', index=False)

In [31]:
totales_diarios_tverde_2022=pd.read_csv('totales_diarios_tverde_2022.csv')

In [32]:
totales_diarios_tverde_2022

,pickup_date,trip_distance,fare_amount,tip_amount,PULocationID,DOLocationID,total_amount,payment_type
0,2008-12-31,5.36,30.50,0.00,267,235,32.30,3.0
1,2009-01-01,15.70,65.50,5.46,727,1135,81.16,8.0
2,2021-12-31,11.41,38.00,3.16,261,338,50.56,4.0
3,2022-01-01,66392.32,20361.61,2493.06,128935,176497,25128.72,1620.0
4,2022-01-02,81294.44,21448.61,2622.52,142460,206993,26851.77,1946.0
...,...,...,...,...,...,...,...,...
363,2022-12-27,8029.92,38372.27,4357.15,209549,296552,49504.87,2859.0
364,2022-12-28,5741.31,37279.94,3925.24,210843,293789,47872.48,2972.0
365,2022-12-29,79490.58,40517.38,4147.34,229134,314744,51686.82,3129.0
366,2022-12-30,6795.41,43623.12,4659.45,229376,315627,55862.47,3290.0


In [36]:
parquet_files_2023 = [file for file in parquet_filesv if 'green_tripdata_2023' in file.name]

In [37]:
parquet_files_2023

[WindowsPath('C:/Users/yopab/Downloads/Datasets/dfexternos/green_tripdata_2023-01.parquet'),
 WindowsPath('C:/Users/yopab/Downloads/Datasets/dfexternos/green_tripdata_2023-02.parquet'),
 WindowsPath('C:/Users/yopab/Downloads/Datasets/dfexternos/green_tripdata_2023-03.parquet'),
 WindowsPath('C:/Users/yopab/Downloads/Datasets/dfexternos/green_tripdata_2023-04.parquet'),
 WindowsPath('C:/Users/yopab/Downloads/Datasets/dfexternos/green_tripdata_2023-05.parquet')]

In [38]:
data_frames_2023 = []

In [39]:
for file in parquet_files_2023:
    df = pd.read_parquet(file)
    data_frames_2023.append(df)

In [40]:
combined_df_2023 = pd.concat(data_frames_2023, ignore_index=True)

In [41]:
# Convertir las columnas de fechas a datetime
combined_df_2023['lpep_pickup_datetime'] = pd.to_datetime(combined_df_2023['lpep_pickup_datetime'])
combined_df_2023['lpep_dropoff_datetime'] = pd.to_datetime(combined_df_2023['lpep_dropoff_datetime'])

In [42]:
combined_df_2023['pickup_date'] = combined_df_2023['lpep_pickup_datetime'].dt.date

In [43]:
daily_totals = combined_df_2023.groupby('pickup_date').agg({
    'trip_distance': 'sum',
    'fare_amount': 'sum',
    'tip_amount': 'sum',
    'PULocationID': 'sum',# Identificador de la ubicación de recogida.
    'DOLocationID': 'sum',#Identificador de la ubicación de destino.
    'total_amount': 'sum',#Monto total pagado por el pasajero.
    'payment_type': 'sum', #Tipo de pago utilizado en el viaje.

}).reset_index()

In [44]:
daily_totals.to_csv('totales_diarios_tverde_2023.csv', index=False)

In [45]:
totales_diarios_tverde_2023=pd.read_csv('totales_diarios_tverde_2023.csv')

In [46]:
totales_diarios_tverde_2023

,pickup_date,trip_distance,fare_amount,tip_amount,PULocationID,DOLocationID,total_amount,payment_type
0,2008-12-31,5.28,36.80,5.95,231,492,51.00,3.0
1,2009-01-01,7.72,34.50,7.20,65,178,43.20,1.0
2,2022-12-09,10.85,41.50,0.00,267,220,52.40,4.0
3,2023-01-01,5191.36,27501.48,3484.45,148312,209734,34885.03,1800.0
4,2023-01-02,4527.00,26414.99,3175.47,147915,219774,33659.66,2061.0
...,...,...,...,...,...,...,...,...
152,2023-05-30,6351.57,39681.02,5116.68,211871,309220,52179.71,2735.0
153,2023-05-31,27116.10,43968.89,5978.87,229445,328318,57927.21,2907.0
154,2023-06-01,3.64,20.50,5.75,255,97,28.75,1.0
155,2023-06-05,2.28,14.20,3.34,129,146,20.04,1.0
